In [26]:
import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
from torchvision.datasets import Omniglot
from torch.utils.data import DataLoader, Subset
from PIL import Image
from prototypical_network import protonet
from data_load_helpers import customized_transform, prepare_data_for_sklearn
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture


In [27]:
# specify the number of classes
num_classes = 50

In [28]:
# import pretrained resnet152 as the embedding function
embedding = torchvision.models.resnet152(weights='DEFAULT')
embedding.eval()
print()

In [29]:
# Define the transformation pipeline
transform = transforms.Compose([
    customized_transform()  # Conditionally convert grayscale images and adjust dimensions
])

# Note that Omniglot has 20 samples per class.
subset_indices = list(range(num_classes*20))  # Select the first 3200 indices

# Load the dataset
fsl_dataset = Omniglot(root='path/to/data', background=False, transform=transform, download=True)
 
subset_dataset = Subset(fsl_dataset, subset_indices)

# Create data loaders
fsl_loader = DataLoader(dataset=subset_dataset, batch_size=20, shuffle=False)

support_data_tensor, support_labels_tensor, query_data_tensor, query_labels_tensor = prepare_data_for_sklearn(fsl_loader)


Files already downloaded and verified


In [30]:
fsl_model = protonet(num_classes = num_classes, embedding = embedding)

# print(fsl_model.prototypes)
# create prototypes.
for images, labels in fsl_loader:
    fsl_model.batch_update_prototype(images[0:5], labels[0:5])

        

In [31]:
# testing phase:
correct_num = 0
for images, labels in fsl_loader:
    output = fsl_model.protonet_fsl(images[5:20])
    print(images[5:20].shape)
    softmax = torch.nn.Softmax(dim=1)
    # print(output)
    # print(labels[0])
    output_label = torch.argmin(softmax(output), dim = 1)
    print(np.sum(output_label.numpy() == (labels[5:20]).numpy()))
    correct_num += np.sum(output_label.numpy() == (labels[5:20]).numpy())
    print(output_label)
    print(labels)
    

torch.Size([15, 3, 224, 224])
12
tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 9, 0, 9, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
torch.Size([15, 3, 224, 224])
13
tensor([1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 3, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
torch.Size([15, 3, 224, 224])
6
tensor([3, 2, 3, 2, 4, 3, 3, 2, 4, 6, 2, 2, 3, 2, 3])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
torch.Size([15, 3, 224, 224])
11
tensor([3, 3, 0, 1, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
torch.Size([15, 3, 224, 224])
5
tensor([4, 4, 4, 2, 4, 6, 6, 6, 4, 6, 6, 6, 2, 6, 2])
tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])
torch.Size([15, 3, 224, 224])
9
tensor([5, 5, 0, 9, 5, 5, 0, 5, 5, 5, 3, 5, 3, 5, 0])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
torch.Size([15, 3, 224, 224])
7
tensor([2, 3, 4, 2, 4, 6, 6, 2, 4, 

In [32]:
print(correct_num/(num_classes * 15))

0.6666666666666666


In [33]:

# Create and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)  # You can choose the number of neighbors
knn.fit(support_data_tensor, support_labels_tensor)



KNeighborsClassifier()

In [34]:
# Predict on the test set
y_pred = knn.predict(query_data_tensor)

# Evaluate the accuracy
accuracy = accuracy_score(query_labels_tensor, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 44.67%


In [35]:
# pip install scikit-learn

In [36]:
len(y_pred)

150

In [23]:
# Initialize and train the logistic regression model
# Set max_iter high enough to ensure convergence, as logistic regression can be slow with large datasets
log_reg = LogisticRegression()  
log_reg.fit(support_data_tensor, support_labels_tensor)

LogisticRegression()

In [24]:
# Make predictions on the test set
y_pred = log_reg.predict(query_data_tensor)

# Evaluate the accuracy
accuracy = accuracy_score(query_labels_tensor, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 62.00%


In [25]:



n_classes = len(np.unique(support_labels_np))

# Initialize a list to hold Gaussian Mixture Models for each class
gmm_models = []

# Fit a GMM for each class
for class_label in range(n_classes):
    # Subset the training data for the current class
    class_data = support_data_np[support_labels_np == class_label]
    
    # Fit a GMM to the data for this class
    gmm = GaussianMixture(n_components=2, covariance_type='diag', random_state=1)  # You can adjust the number of components
    gmm.fit(class_data)
    
    # Append the trained GMM model to the list
    gmm_models.append(gmm)

# Perform classification on the test data
y_pred = []
for test_sample in query_data_np:
    # Compute the likelihood of the test sample under each class's GMM
    likelihoods = [gmm.score_samples(test_sample.reshape(1, -1)) for gmm in gmm_models]
    
    # Assign the class with the highest likelihood
    predicted_class = np.argmax(likelihoods)
    y_pred.append(predicted_class)

# Convert predictions to numpy array
y_pred = np.array(y_pred)

# Evaluate the accuracy
accuracy = accuracy_score(query_labels_np, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 58.00%
